# Mermaid Diagram Examples

Test `graph.to_mermaid()` rendering. Uses the native `text/vnd.mermaid` MIME type — no CDN, no external services. Requires JupyterLab 4.1+ or Notebook 7.1+.

In [ ]:
from hypergraph import Graph, node, route, ifelse, END

## 1. Simple Pipeline

In [ ]:
@node(output_name="cleaned")
def clean(raw_text: str) -> str:
    return raw_text.strip().lower()


@node(output_name="tokens")
def tokenize(cleaned: str) -> list[str]:
    return cleaned.split()


@node(output_name="embedding")
def embed(tokens: list[str]) -> list[float]:
    return [0.1] * len(tokens)


pipeline = Graph(nodes=[clean, tokenize, embed])
pipeline.to_mermaid()

## 2. With Type Annotations

In [ ]:
pipeline.to_mermaid(show_types=True)

## 3. Branching (ifelse)

In [ ]:
@node(output_name="cached_result")
def use_cache(query: str) -> str:
    return "cached answer"


@node(output_name="fresh_result")
def compute_fresh(query: str) -> str:
    return "fresh answer"


@ifelse(when_true="use_cache", when_false="compute_fresh")
def check_cache(query: str) -> bool:
    return query in ["hello", "world"]


Graph(nodes=[check_cache, use_cache, compute_fresh]).to_mermaid()

## 4. Agentic Loop (cycle with END)

In [ ]:
@node(output_name="response")
def generate(messages: list[dict]) -> str:
    return "I can help with that!"


@node(output_name="messages")
def accumulate(messages: list[dict], response: str) -> list[dict]:
    return messages + [{"role": "assistant", "content": response}]


@route(targets=["generate", END])
def should_continue(messages: list[dict]) -> str:
    if len(messages) > 5:
        return END
    return "generate"


Graph(nodes=[generate, accumulate, should_continue]).to_mermaid()

## 5. Nested Graph (collapsed vs expanded)

In [ ]:
@node(output_name="cleaned")
def clean_text(text: str) -> str:
    return text.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    return cleaned.lower()


preprocess = Graph(nodes=[clean_text, normalize], name="preprocess")


@node(output_name="result")
def analyze(normalized: str) -> dict:
    return {"length": len(normalized)}


workflow = Graph(nodes=[preprocess.as_node(), analyze])
workflow.to_mermaid(depth=0)

In [ ]:
workflow.to_mermaid(depth=1)

## 6. Direction and Separate Outputs

In [ ]:
pipeline.to_mermaid(direction="LR")

In [ ]:
pipeline.to_mermaid(separate_outputs=True)

## 7. Raw Source

`print()` gives you the raw Mermaid text — paste into any Mermaid-compatible tool.

In [ ]:
print(pipeline.to_mermaid())